# 14.7 词的相似性和类比任务
- **目录**
  - 14.7.1 加载预训练词向量
  - 14.7.2 应用预训练词向量
    - 14.7.2.1 词相似度
    - 14.7.2.2 词类比

-  在14.4节中，我们在一个小的数据集上训练了一个word2vec模型，并使用它为一个输入词寻找语义相似的词。
- 实际上，在**大型语料库上预先训练的词向量**可以应用于下游的自然语言处理任务，这将在后面的15章中讨论。
- 为了直观地演示大型语料库中预训练词向量的语义，让我们将预训练词向量应用到词的**相似性**和**类比**任务中。

In [1]:
%matplotlib inline
import os
import torch
from torch import nn
from d2l import torch as d2l

## 14.7.1 加载预训练词向量

- 以下列出维度为50、100和300的预训练GloVe嵌入，可从[GloVe网站](https://nlp.stanford.edu/projects/glove/)下载。
- 预训练的fastText嵌入有多种语言。
  - 可以从[fastText网站](https://fasttext.cc/)下载300维度的英文版本（“wiki.en”）。

In [6]:
## 本节使用第一个GloVe嵌入
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

- 为了加载这些预训练的GloVe和fastText嵌入，我们定义了以下`TokenEmbedding`类。


In [2]:
class TokenEmbedding:
    """GloVe嵌入"""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe网站：https://nlp.stanford.edu/projects/glove/
        # fastText网站：https://fasttext.cc/
        ##注意此处要加上encoding='utf-8'参数，否则会报错
        with open(os.path.join(data_dir, 'vec.txt'), 'r', encoding='utf-8') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # 跳过标题信息，例如fastText中的首行
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

- TokenEmbedding完整注释

In [3]:
#@save

class TokenEmbedding:
    """GloVe嵌入"""
    '''
   （1）idx_to_token：通过索引获取词元。
   （2） idx_to_vec：通过索引获取词向量。
   （3）token_to_idx：通过词元获取词元索引。
   （4）unknown_idx：未知词元索引，比如'<unk>'词元。
    '''
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}
        
    '''
    （1）_load_embedding方法：通过GloVe嵌入的名称(embedding_name参数)获取词向量，在本例中是使用glove.6b.50d。        
    '''
    def _load_embedding(self, embedding_name):
        '''
        （1）idx_to_token进行初始化将未知词元'<unk>'放在第1个。
        （2）idx_to_vec初始化为空列表。
        （3）此处下载glove.6b.50d词嵌入，保存在../data/golve.6B.50d/vec.txt。
            类似：
                 world -0.41486 0.71848 -0.3045 0.87445 0.22441 -0.56488 -0.37566 -0.44801 0.61347 -0.11359 0.74556  ...
                 can 0.8052 0.37121 0.55933 -0.011405 0.17319 0.195 0.057701 -0.12447 -0.011342 0.20654 0.41079 ...
        '''
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe网站：https://nlp.stanford.edu/projects/glove/
        # fastText网站：https://fasttext.cc/
        ##注意此处要加上encoding='utf-8'参数，否则会报错
        with open(os.path.join(data_dir, 'vec.txt'), 'r', encoding='utf-8') as f:
            # 迭代词向量文件的每一行
            for line in f:
                # 将对每一行按照空格进行分割
                elems = line.rstrip().split(' ')
               ''' 
               第一列是词元字符串，后面50列是该词元对应的词向量。
               '''
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                '''
                注意：idx_to_vec是一个嵌套列表，每个子列表是某个特定词元的词向量。
                '''
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
         # 未知词元'<unk>'的词向量保存在idx_to_vec第一个元素（下标为0）
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        # 将idx_to_vec转换成张量类型
        return idx_to_token, torch.tensor(idx_to_vec)
        
    # 实现词向量的[]索引机制
    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

- 下面加载50维GloVe嵌入（在维基百科的子集上预训练）。
- 创建`TokenEmbedding`实例时，如果尚未下载指定的嵌入文件，则必须下载该文件。


In [4]:
d2l.download_extract('glove.6b.50d')

'..\\data\\glove.6B.50d'

In [5]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

- 输出词表大小。词表包含400000个词（词元）和一个特殊的未知词元。

In [6]:
len(glove_6b50d)

400001

- 下面可以得到词表中一个单词的索引，反之亦然。


In [7]:
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')

In [8]:
# 某个词元的词向量
glove_6b50d.idx_to_vec[:2]

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
         -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
          2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
          1.1658e-02,  1.0204e-01, -1

In [9]:
# 余弦相似度计算
a = glove_6b50d.idx_to_vec[glove_6b50d.token_to_idx['chip']]
b = glove_6b50d.idx_to_vec[glove_6b50d.token_to_idx['intel']]
a@b/(torch.norm(a)*torch.norm(b))

tensor(0.7491)

## 14.7.2 应用预训练词向量

- 使用加载的GloVe向量，我们将通过下面的词相似性和类比任务中来展示词向量的语义。

### 14.7.2.1 词相似度

- 与 14.4.3节类似，为了根据词向量之间的余弦相似性为输入词查找语义相似的词，我们实现了以下`knn`（$k$近邻）函数。


In [10]:
def knn(W, x, k):
    # 增加1e-9以获得数值稳定性
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

- 然后使用`TokenEmbedding`的实例`embed`中预训练好的词向量来搜索相似的词。


In [11]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # 排除输入词
        print(f'{embed.idx_to_token[int(i)]}：cosine相似度={float(c):.3f}')

- `glove_6b50d`中预训练词向量的词表包含400000个词和一个特殊的未知词元。
- 排除输入词和未知词元后，下面代码在词表中找到与“chip”一词语义最相似的三个词。


In [12]:
get_similar_tokens('chip', 3, glove_6b50d)

chips：cosine相似度=0.856
intel：cosine相似度=0.749
electronics：cosine相似度=0.749


- 下面输出与“baby”和“beautiful”相似的词。


In [13]:
get_similar_tokens('baby', 3, glove_6b50d)

babies：cosine相似度=0.839
boy：cosine相似度=0.800
girl：cosine相似度=0.792


In [14]:
get_similar_tokens('beautiful', 3, glove_6b50d)

lovely：cosine相似度=0.921
gorgeous：cosine相似度=0.893
wonderful：cosine相似度=0.830


### 14.7.2.2 词类比(analogy)

- 除了找到相似的词，还可以将词向量应用到词类比任务中，例如：
  - “man” : “woman” :: “son” : “daughter”是一个词的类比。
  - “man”是对“woman”的类比，“son”是对“daughter”的类比。
- 具体来说，词类比任务可以定义为：
- 对于单词类比$a : b :: c : d$，给出前三个词$a$、$b$和$c$，找到$d$。
  - 用$\text{vec}(w)$表示词$w$的向量，
  - 为了完成这个类比，此处将找到一个词，其向量与$\text{vec}(c)+\text{vec}(b)-\text{vec}(a)$的结果最相似。


In [15]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[int(topk[0])]  # 删除未知词

- 使用加载的词向量来验证“male-female”类比。


In [16]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

- 下面完成一个“首都-国家”的类比：
  - “beijing” : “china” :: “tokyo” : “japan”。
- 该例子说明了预训练词向量中的语义。


In [17]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

- 另外，对于“bad” : “worst” :: “big” : “biggest”等“形容词-形容词最高级”的比喻，预训练词向量可以捕捉到句法信息。

In [18]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

- 为了演示在预训练词向量中捕捉到的过去式概念，可以使用“现在式-过去式”的类比来测试句法：“do” : “did” :: “go” : “went”。


In [19]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'

## 小结

* 在实践中，在大型语料库上预先训练的词向量可以应用于下游的自然语言处理任务。
* 预训练的词向量可以应用于词的相似性和类比任务。